<a href="https://colab.research.google.com/github/finnagin/BayesEnsemble/blob/master/CS_536_Project_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install torch torchvision
!pip3 install transformers
!pip3 install --upgrade sultan
!pip install simpletransformers

!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir ./apex/

!git clone https://github.com/finnagin/BayesEnsemble

import torch
import transformers
from transformers import RobertaTokenizer, BertPreTrainedModel
from sklearn.feature_extraction.text import TfidfVectorizer

import math
import random
import requests
from sultan.api import Sultan
import pandas as pd
import os
from google.colab import drive
import numpy as np
import zipfile
import gzip
import rpy2.robjects as robjects
import io
import requests
import torch.nn.functional as F
from simpletransformers.classification import ClassificationModel
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
s = Sultan()

     |████████████████████████████████| 501kB 3.5MB/s 
     |████████████████████████████████| 3.7MB 51.1MB/s 
     |████████████████████████████████| 1.0MB 40.5MB/s 
     |████████████████████████████████| 870kB 38.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=fc5dc8783e5f08cec35b2a58f6b72e858de29321c66a54dd271b819071ee153c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 153kB 3.5MB/s 
     |████████████████████████████████| 204kB 11.0MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=951fcb333ceebad007be6734efa8f9f4f996919bcf53adeff9ebc1d2847269ee
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
Cloning into 'apex'...
remote: Enumerating objects: 251, done.
remote: Counting o

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
s140_file_url = 'http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip'
req_s140 = requests.get(s140_file_url)
zip_s140 = zipfile.ZipFile(io.BytesIO(req_s140.content))

df_s140_train = pd.read_csv(zip_s140.open('training.1600000.processed.noemoticon.csv'), encoding = "ISO-8859-1", names = ['sentiment', 'id', 'date', 'query', 'user', 'text'], index_col=False)
df_s140_test = pd.read_csv(zip_s140.open('testdata.manual.2009.06.14.csv'), encoding = "ISO-8859-1", names = ['sentiment', 'id', 'date', 'query', 'user', 'text'], index_col=False)

def prepare_s140(data):
  preped_data = data[data.sentiment != 2]
  preped_data['sentiment'] = np.where(preped_data['sentiment'] == 4, 1, preped_data['sentiment'])
  return preped_data

df_s140_train = prepare_s140(df_s140_train)
df_s140_test = prepare_s140(df_s140_test)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [0]:
def prepare_imdb(data):
  prepared_data = data.copy()
  prepared_data['sentiment'] = np.where(prepared_data['sentiment'] == 'Negative', 0, prepared_data['sentiment'])
  prepared_data['sentiment'] = np.where(prepared_data['sentiment'] == 'Positive', 1, prepared_data['sentiment'])
  return prepared_data

# Dataset originally proposed by: https://arxiv.org/abs/1909.12434
if not os.path.exists("/content/counterfactually-augmented-data"):
  s.git('clone', 'https://github.com/dkaushik96/counterfactually-augmented-data.git').run()

s.sed('-i -e \'s/this\! 3011/this\!	3011/g\' /content/counterfactually-augmented-data/sentiment/combined/paired/train_paired.tsv').run()

df_imdb_train = prepare_imdb(pd.read_csv('/content/counterfactually-augmented-data/sentiment/orig/eighty_percent/train.tsv', sep='\t', names=('sentiment', 'text', 'id'), skiprows=[0]))
df_imdb_test = prepare_imdb(pd.read_csv('/content/counterfactually-augmented-data/sentiment/orig/eighty_percent/test.tsv', sep='\t', names=('sentiment', 'text', 'id'), skiprows=[0]))

In [0]:
df_amazon_test = pd.read_csv('/content/BayesEnsemble/data/amazon_cells_labelled.txt', sep='\t', header=None, names = ['text', 'sentiment'])
df_yelp_test = pd.read_csv('/content/BayesEnsemble/data/yelp_labelled.txt', sep='\t', header=None, names = ['text', 'sentiment'])


In [0]:
num_bayes_nets = 30


# Adapted from HuggingFace's RobertaForMultipleChoice
# https://huggingface.co/transformers/_modules/transformers/modeling_roberta.html#RobertaForMultipleChoice

class RobertaForBayesianContextSM(transformers.BertPreTrainedModel):
    config_class = transformers.RobertaConfig
    pretrained_model_archive_map = transformers.ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)

        self.roberta = transformers.RobertaModel(config)
        self.num_bayes_nets = num_bayes_nets
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, self.num_bayes_nets)
        self.combiner = torch.nn.Linear(self.num_bayes_nets, 2)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        token_type_ids=None,
        attention_mask=None,
        labels=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        bayes_nets_outputs=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the multiple choice classification loss.
            Indices should be in ``[0, ..., num_choices]`` where `num_choices` is the size of the second dimension
            of the input tensors. (see `input_ids` above)

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.RobertaConfig`)
         and inputs:
        loss (:obj:`torch.FloatTensor`` of shape ``(1,)`, `optional`, returned when :obj:`labels` is provided):
            Classification loss.
        classification_scores (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, num_choices)`):
            `num_choices` is the second dimension of the input tensors. (see `input_ids` above).

            Classification scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import RobertaTokenizer, RobertaForMultipleChoice
        import torch

        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForMultipleChoice.from_pretrained('roberta-base')
        choices = ["Hello, my dog is cute", "Hello, my cat is amazing"]
        input_ids = torch.tensor([tokenizer.encode(s, add_special_tokens=True) for s in choices]).unsqueeze(0)  # Batch size 1, 2 choices
        labels = torch.tensor(1).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, classification_scores = outputs[:2]

        """
        num_choices = 2

        flat_input_ids = input_ids.view(-1, input_ids.size(-1))
        flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
        flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
        flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
        outputs = self.roberta(
            flat_input_ids,
            position_ids=flat_position_ids,
            token_type_ids=flat_token_type_ids,
            attention_mask=flat_attention_mask,
            head_mask=head_mask,
        )
        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        bayes_weights = self.classifier(pooled_output)
        bayes_weights = torch.nn.functional.softmax(bayes_weights, dim=1)
        
        self.combiner.weight.data = torch.tensor(bayes_nets_outputs, dtype=torch.float).cuda()
        logits = self.combiner(bayes_weights)
        reshaped_logits = logits.view(-1, num_choices)
        #if random.randint(0, 200) == 10:
        #    print("Bayes weights", bayes_weights)
        #    print("Bayes probabilities", self.combiner.weight.data)
        #    print("Logits", reshaped_logits)
        #    print("Label", labels.item())

        outputs = (reshaped_logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)
            outputs = (loss,) + outputs

        return outputs  # (loss), reshaped_logits, (hidden_states), (attentions)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForBayesianContextSM.from_pretrained('roberta-base')
model.cuda()
''

''

In [0]:
#from sklearn.feature_extraction.text import TfidfVectorizer

imdb_size = len(df_imdb_test) + len(df_imdb_train)

s140_sample = df_s140_train.head(imdb_size).append(df_s140_train.tail(imdb_size))

all_text_sample = df_imdb_train['text'].append(df_imdb_test['text']).append(s140_sample['text']).values
all_imdb = df_imdb_train['text'].append(df_imdb_test['text']).values



vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.9,
                                 stop_words='english', strip_accents='unicode', min_df=530) #530

X_train = vectorizer.fit_transform(all_text_sample)


feature_names = vectorizer.get_feature_names()

if feature_names:
    feature_names = np.asarray(feature_names)


In [0]:
def text_list_to_pandas_nums(data, vectorizer):
    data = data.reset_index()
    text_list = list(data['text'])
    sparse_scipy = vectorizer.transform(text_list)
    dense_numpy = sparse_scipy.toarray()
    pandas_df = pd.DataFrame(dense_numpy) * 4
    pandas_df = np.ceil(pandas_df).astype(int)
    pandas_df['class'] = data['sentiment']
    return pandas_df

save = False

randomized_s140 = df_s140_train.sample(frac=1, random_state=0)
randomized_imdb = df_imdb_test.sample(frac=1, random_state=0)

df_ST_s140_train_bayes = randomized_s140[['sentiment', 'text', 'id']].head(4 * len(df_imdb_train))
df_ST_s140_train_roberta = randomized_s140[['sentiment', 'text', 'id']].tail(4 * len(df_imdb_train))

df_ST_imdb_train_bayes = df_imdb_train[['sentiment', 'text', 'id']].sample(frac=1, random_state=0)
df_ST_imdb_train_roberta = randomized_imdb[['sentiment', 'text', 'id']].head(19000)

df_ST_imdb_test = randomized_imdb[['sentiment', 'text', 'id']].tail(1000)
df_ST_s140_test = randomized_s140[['sentiment', 'text', 'id']][100000 : 101000]


df_amazon_test = pd.read_csv('/content/BayesEnsemble/data/amazon_cells_labelled.txt', sep='\t', header=None, names = ['text', 'sentiment'])
#df_amazon_test['id'] = df_amazon_test.index.values

df_yelp_test = pd.read_csv('/content/BayesEnsemble/data/yelp_labelled.txt', sep='\t', header=None, names = ['text', 'sentiment'])
#df_yelp_test['id'] = df_yelp_test.index.values


# Now need to extract features for sentiment140, IMDB reviews train / test
# And also for our yelp/amazon testing data distribution.


pdframe_s140_train_bayes = text_list_to_pandas_nums(df_ST_s140_train_bayes, vectorizer)
pdframe_s140_train_roberta = text_list_to_pandas_nums(df_ST_s140_train_roberta, vectorizer)
pdframe_imdb_train_bayes = text_list_to_pandas_nums(df_ST_imdb_train_bayes, vectorizer)
pdframe_imdb_train_roberta = text_list_to_pandas_nums(df_ST_imdb_train_roberta, vectorizer)

pdframe_imdb_test = text_list_to_pandas_nums(df_ST_imdb_test, vectorizer)
pdframe_s140_test = text_list_to_pandas_nums(df_ST_s140_test, vectorizer)
pdframe_yelp_test = text_list_to_pandas_nums(df_yelp_test, vectorizer)
pdframe_amazon_test = text_list_to_pandas_nums(df_amazon_test, vectorizer)


pdframe_s140_train_bayes['id'] = df_ST_s140_train_bayes.index.values
pdframe_s140_train_roberta['id'] = df_ST_s140_train_roberta.index.values
pdframe_imdb_train_bayes['id'] = df_ST_imdb_train_bayes.index.values
pdframe_imdb_train_roberta['id'] = df_ST_imdb_train_roberta.index.values

pdframe_imdb_test['id'] = df_ST_imdb_test.index.values
pdframe_s140_test['id'] = df_ST_s140_test.index.values
pdframe_yelp_test['id'] = df_yelp_test.index.values
pdframe_amazon_test['id'] = df_amazon_test.index.values

if save:
    pdframe_s140_train_bayes.to_csv('tokenized_bayes_train_from_s140_file_.csv', header=False, index=False)
    pdframe_s140_train_roberta.to_csv('tokenized_roberta_train_from_s140_file_.csv', header=False, index=False)
    pdframe_imdb_train_bayes.to_csv('tokenized_bayes_train_from_imdb_file_.csv', header=False, index=False)
    pdframe_imdb_train_roberta.to_csv('tokenized_roberta_train_from_imdb_file_.csv', header=False, index=False)

    pdframe_imdb_test.to_csv('tokenized_test_from_imdb_file_.csv', header=False, index=False)
    pdframe_s140_test.to_csv('tokenized_test_from_s140_file_.csv', header=False, index=False)
    pdframe_yelp_test.to_csv('tokenized_test_from_yelp_file_.csv', header=False, index=False)
    pdframe_amazon_test.to_csv('tokenized_test_from_amazon_file_.csv', header=False, index=False)

'''
splits = 5
for i in range(0, splits):
    s140_frac = int(len(df_ST_s140_train) / splits)
    imdb_frac = int(len(df_ST_imdb_train) / splits)

    current_s140 = df_ST_s140_train[i * s140_frac : (i+1) * s140_frac]
    current_imdb = df_ST_imdb_train[i * imdb_frac : (i+1) * imdb_frac]

    pdframe = text_list_to_pandas_nums(current_imdb.append(current_s140), vectorizer)
    pdframe.to_csv('tokenized_train_from_imdb_and_s140_file_' + str(i+1) + '.csv')

'''

"\nsplits = 5\nfor i in range(0, splits):\n    s140_frac = int(len(df_ST_s140_train) / splits)\n    imdb_frac = int(len(df_ST_imdb_train) / splits)\n\n    current_s140 = df_ST_s140_train[i * s140_frac : (i+1) * s140_frac]\n    current_imdb = df_ST_imdb_train[i * imdb_frac : (i+1) * imdb_frac]\n\n    pdframe = text_list_to_pandas_nums(current_imdb.append(current_s140), vectorizer)\n    pdframe.to_csv('tokenized_train_from_imdb_and_s140_file_' + str(i+1) + '.csv')\n\n"

In [0]:
def get_prob_tensors(i, lr_data, naive_data, tan_data, data):
    lr_probs = lr_data.iloc[i]
    naive_probs = naive_data.iloc[i]
    tan_probs = tan_data.iloc[i]
    
    text = data.iloc[i]['text']

    bayes_nets_p_positive_lr = np.reshape(lr_probs.values[1:], (1, -1))
    bayes_nets_p_positive_naive = np.reshape(naive_probs.values[1:], (1, -1))
    bayes_nets_p_positive_tan = np.reshape(tan_probs.values[1:], (1, -1))

    all_positive = np.append(np.append(bayes_nets_p_positive_lr, bayes_nets_p_positive_naive), bayes_nets_p_positive_tan)
    all_positive = np.reshape(all_positive, (1, -1))
    all_negative = np.reshape(1 - all_positive, (1, -1))
    
    bayes_nets_outputs = np.append(all_positive, all_negative, axis=0)
    return bayes_nets_outputs



df_lr_output_s140_roberta_train = pd.read_csv('/content/BayesEnsemble/data/lr/roberta_s140_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_output_s140_roberta_train = pd.read_csv('/content/BayesEnsemble/data/naive/roberta_s140_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_output_s140_roberta_train = pd.read_csv('/content/BayesEnsemble/data/tan/roberta_s140_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

df_lr_amazon_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/amazon_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_amazon_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/amazon_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_amazon_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/amazon_prob_tan.csv', header=0).drop(columns='Unnamed: 0')


df_lr_yelp_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/yelp_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_yelp_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/yelp_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_yelp_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/yelp_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

df_lr_s140_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/s140_test_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_s140_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/s140_test_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_s140_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/s140_test_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

df_lr_imdb_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/imdb_test_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_imdb_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/imdb_test_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_imdb_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/imdb_test_prob_tan.csv', header=0).drop(columns='Unnamed: 0')


In [0]:
def get_prob_tensors(i, lr_data, naive_data, tan_data, data):
    lr_probs = lr_data.iloc[i]
    naive_probs = naive_data.iloc[i]
    tan_probs = tan_data.iloc[i]
    
    text = data.iloc[i]['text']

    bayes_nets_p_positive_lr = np.reshape(lr_probs.values[1:], (1, -1))
    bayes_nets_p_positive_naive = np.reshape(naive_probs.values[1:], (1, -1))
    bayes_nets_p_positive_tan = np.reshape(tan_probs.values[1:], (1, -1))

    all_positive = np.append(np.append(bayes_nets_p_positive_lr, bayes_nets_p_positive_naive), bayes_nets_p_positive_tan)
    all_positive = np.reshape(all_positive, (1, -1))
    all_negative = np.reshape(1 - all_positive, (1, -1))
    
    bayes_nets_outputs = np.append(all_positive, all_negative, axis=0)
    return bayes_nets_outputs



df_lr_output_s140_roberta_train = pd.read_csv('/content/BayesEnsemble/data/lr/roberta_s140_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_output_s140_roberta_train = pd.read_csv('/content/BayesEnsemble/data/naive/roberta_s140_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_output_s140_roberta_train = pd.read_csv('/content/BayesEnsemble/data/tan/roberta_s140_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

df_lr_amazon_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/amazon_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_amazon_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/amazon_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_amazon_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/amazon_prob_tan.csv', header=0).drop(columns='Unnamed: 0')


df_lr_yelp_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/yelp_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_yelp_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/yelp_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_yelp_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/yelp_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

df_lr_s140_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/s140_test_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_s140_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/s140_test_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_s140_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/s140_test_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

df_lr_imdb_output_test = pd.read_csv('/content/BayesEnsemble/data/lr/imdb_test_prob_lr.csv', header=0).drop(columns='Unnamed: 0')
df_naive_imdb_output_test = pd.read_csv('/content/BayesEnsemble/data/naive/imdb_test_prob_naive.csv', header=0).drop(columns='Unnamed: 0')
df_tan_imdb_output_test = pd.read_csv('/content/BayesEnsemble/data/tan/imdb_test_prob_tan.csv', header=0).drop(columns='Unnamed: 0')

opt = torch.optim.Adam(model.parameters(), lr=0.00003)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, 20, eta_min=0, last_epoch=-1)


n_steps_til_test = 1000
grad_accumulation_steps = 500

ensemble_train_lr = df_lr_output_s140_roberta_train
ensemble_train_naive = df_naive_output_s140_roberta_train
ensemble_train_tan = df_tan_output_s140_roberta_train

dataset_train = df_ST_s140_train_roberta

'''
ensemble_test_lr = df_lr_s140_output_test
ensemble_test_naive = df_naive_s140_output_test
ensemble_test_tan = df_tan_s140_output_test

dataset_test = df_ST_s140_test
'''

ensemble_test_lr = df_lr_yelp_output_test
ensemble_test_naive = df_naive_yelp_output_test
ensemble_test_tan = df_tan_yelp_output_test

dataset_test = df_yelp_test

epochs = 1



for epoch in range(0, epochs):
    print("\nEpoch = " + str(epoch) + '\n')
    for i in range(0, 70000):#len(dataset_train)):
        if i % n_steps_til_test == 0:
            print("Testing after " + str(i) + " steps.")
            with torch.no_grad():
                total_loss = 0
                total_correct = 0
                for j in range(0, len(dataset_test)):
                    #datapoint = ensemble_test.iloc[j]
                    text = dataset_test.iloc[j]['text']
                    
                    #bayes_nets_p_positive = np.reshape(datapoint.values[1:], (1, -1))
                    #bayes_nets_p_negative = np.reshape(1 - bayes_nets_p_positive, (1, -1))
                    #bayes_nets_outputs = np.append(bayes_nets_p_positive, bayes_nets_p_negative, axis=0)

                    bayes_nets_outputs = get_prob_tensors(j, ensemble_test_lr, ensemble_test_naive, ensemble_test_tan, dataset_test)
                    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0).cuda()
                    label = dataset_test.iloc[j]['sentiment']
                    label_tensor = torch.tensor(label).unsqueeze(0).cuda()
                    outputs = model(input_ids, labels=label_tensor, bayes_nets_outputs=bayes_nets_outputs)
                    loss, classification_scores = outputs[:2]
                    probs = torch.nn.functional.softmax(classification_scores, dim=1)
                    if probs[0][0] > 0.5 and label == 0:
                        total_correct += 1
                    if probs[0][1] > 0.5 and label == 1:
                        total_correct += 1
                    total_loss += loss.item()
                print("\n\nAverage loss = " + str(total_loss / len(dataset_test)))
                print("Fraction correct = " + str(total_correct / len(dataset_test)) + '\n\n')
        #datapoint = ensemble_train.iloc[i]
        text = dataset_train.iloc[i]['text']

        #bayes_nets_p_positive = np.reshape(datapoint.values[1:], (1, -1))
        #bayes_nets_p_negative = np.reshape(1 - bayes_nets_p_positive, (1, -1))
        #bayes_nets_outputs = np.append(bayes_nets_p_positive, bayes_nets_p_negative, axis=0)

        bayes_nets_outputs = get_prob_tensors(i, ensemble_train_lr, ensemble_train_naive, ensemble_train_tan, dataset_train)

        input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0).cuda()
        label = dataset_train.iloc[i]['sentiment']
        label_tensor = torch.tensor(label).unsqueeze(0).cuda()
        outputs = model(input_ids, labels=label_tensor, bayes_nets_outputs=bayes_nets_outputs)
        loss, classification_scores = outputs[:2]
        probs = torch.nn.functional.softmax(classification_scores, dim=1)
        #print(i, ':', label, probs)

        #print("Loss:", loss.item(), '\n\n')
        loss.backward()
        if (i + 1) % grad_accumulation_steps == 0:
            loss = loss / grad_accumulation_steps
            opt.step()
            lr_scheduler.step()
            opt.zero_grad()




Epoch = 0

Testing after 0 steps.


Average loss = 0.8249019944071769
Fraction correct = 0.304


Testing after 1000 steps.


Average loss = 0.824433462381363
Fraction correct = 0.301


Testing after 2000 steps.


Average loss = 0.8237083000540734
Fraction correct = 0.31




In [0]:

ensemble_test_lr = df_lr_s140_output_test
ensemble_test_naive = df_naive_s140_output_test
ensemble_test_tan = df_tan_s140_output_test

dataset_test = df_ST_s140_test.sample(frac=1)


with torch.no_grad():
    total_loss = 0
    total_correct = 0
    for j in range(0, len(dataset_test)):
        text = dataset_test.iloc[j]['text']
        bayes_nets_outputs = get_prob_tensors(j, ensemble_test_lr, ensemble_test_naive, ensemble_test_tan, dataset_test)
        input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0).cuda()
        label = dataset_test.iloc[j]['sentiment']
        label_tensor = torch.tensor(label).unsqueeze(0).cuda()
        outputs = model(input_ids, labels=label_tensor, bayes_nets_outputs=bayes_nets_outputs)
        loss, classification_scores = outputs[:2]
        probs = torch.nn.functional.softmax(classification_scores, dim=1)
        if probs[0][0] > 0.5 and label == 0:
            total_correct += 1
        if probs[0][1] > 0.5 and label == 1:
            total_correct += 1
        total_loss += loss.item()
    print("\n\nAverage loss = " + str(total_loss / len(dataset_test)))
    print("Fraction correct = " + str(total_correct / len(dataset_test)) + '\n\n')



Average loss = 0.4573885177727789
Fraction correct = 0.786




In [0]:
torch.save(model, "/content/drive/My Drive/CS 536 Models/roBERTa_s140_30_probs_train_better.pt")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RobertaForBayesianContext. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

total_correct_by_mass = 0
total_correct_by_vote = 0

for j in range(0, len(dataset_test)):
    n_models = 30
    bound = n_models / 2
    bayes_nets_p_positive = get_prob_tensors(j, ensemble_test_lr, ensemble_test_naive, ensemble_test_tan, dataset_test)[0]
    label = dataset_test.iloc[j]['sentiment']
    total_p_positive = np.sum(bayes_nets_p_positive)

    models_predicting_positive = len(bayes_nets_p_positive[bayes_nets_p_positive > 0.5])


    if total_p_positive < bound and label == 0:
        total_correct_by_mass += 1
    if total_p_positive > bound and label == 1:
        total_correct_by_mass += 1
    
    if models_predicting_positive < bound and label == 0:
        total_correct_by_vote += 1
    if models_predicting_positive > bound and label == 1:
        total_correct_by_vote += 1

    if models_predicting_positive == bound and total_p_positive < bound and label == 0:
        total_correct_by_vote += 1
    if models_predicting_positive == bound and total_p_positive > bound and label == 1:
        total_correct_by_vote += 1
print("Fraction correct by mass = " + str(total_correct_by_mass / len(dataset_test)))
print("Fraction correct by vote = " + str(total_correct_by_vote / len(dataset_test)))

Fraction correct by mass = 0.683
Fraction correct by vote = 0.69


In [0]:
config = {
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "num_train_epochs": 5,
    "evaluate_during_training": True,
    "save_model_every_epoch": False,
    "no_cache": True,
}


deep_classifier = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=config)
!rm -rf /content/outputs


deep_classifier.train_model(args=config, train_df=df_amazon_test, eval_df=df_yelp_test)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:249: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.692778

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.794202Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.482630Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.350403

...

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:587: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 1.314598

Running loss: 0.008587

Running loss: 0.001969

Running loss: 0.001538

In [0]:
class linear(torch.nn.Module):
    def __init__(self):
        super(linear, self).__init__()
        self.linear = torch.nn.Linear(30, 2)

    def forward(self, x):
        x = self.linear(x)
        return x


linear = linear().cuda()

loss_funct = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(linear.parameters(), lr=0.0001)
total_loss = 0

try:
    type(bayes_batch)
except:
    bayes_batch = torch.tensor(get_prob_tensors(0, ensemble_train_lr, ensemble_train_naive, ensemble_train_tan, dataset_train)[0])
    bayes_batch = torch.reshape(bayes_batch, (1, 30)).type(torch.float32)
    for j in range(1, len(ensemble_train_lr)):
        bayes_nets_outputs = torch.tensor(get_prob_tensors(j, ensemble_train_lr, ensemble_train_naive, ensemble_train_tan, dataset_train)[0])
        bayes_nets_outputs = torch.reshape(bayes_nets_outputs, (1, 30)).type(torch.float32)
        bayes_batch = torch.cat((bayes_batch, bayes_nets_outputs), dim=0)
    print(bayes_batch.size())
labels = torch.tensor((dataset_train['sentiment'].values).astype(int)).type(torch.long)

labels = labels.cuda()
bayes_batch = bayes_batch.cuda()

bayes_batch_test = torch.tensor(get_prob_tensors(0, ensemble_test_lr, ensemble_test_naive, ensemble_test_tan, dataset_test)[0])
bayes_batch_test = torch.reshape(bayes_batch_test, (1, 30)).type(torch.float32)
for j in range(1, len(ensemble_test_lr)):
    bayes_nets_outputs_test = torch.tensor(get_prob_tensors(j, ensemble_test_lr, ensemble_test_naive, ensemble_test_tan, dataset_test)[0])
    bayes_nets_outputs_test = torch.reshape(bayes_nets_outputs_test, (1, 30)).type(torch.float32)
    bayes_batch_test = torch.cat((bayes_batch_test, bayes_nets_outputs_test), dim=0)

labels_test = torch.tensor((dataset_test['sentiment'].values).astype(int)).type(torch.long)
labels_test = labels_test.cuda()
bayes_batch_test = bayes_batch_test.cuda()



test_accuracies = []
train_losses = []   

for epoch in range(0, 10000):
    running_loss = 0.0
    optimizer.zero_grad()
    prediction = linear(bayes_batch)
    loss = loss_funct(prediction, labels)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 499:
        n_correct = 0
        print("Loss", loss.item())
        with torch.no_grad():
            train_losses.append(loss.item())
            prediction_probs = torch.nn.functional.softmax(linear(bayes_batch_test), dim=1)[:,1]
            for k in range(0, len(prediction_probs)):
                if labels_test[k].item() == 1 and prediction_probs[k].item() > 0.5:
                    n_correct += 1
                if labels_test[k].item() == 0 and prediction_probs[k].item() < 0.5:
                    n_correct += 1
        accuracy = n_correct / len(prediction_probs)
        test_accuracies.append(accuracy)
        print("Accuracy", accuracy)


torch.Size([77048, 30])


TypeError: ignored

In [0]:
list(linear.parameters())

[Parameter containing:
 tensor([[ 5.8821e-02, -2.2054e-02,  4.0293e-03, -2.6728e-01, -3.5615e-01,
          -3.0579e-01, -1.6866e-01, -1.1805e-01, -2.5856e-01, -9.5296e-02,
           2.1059e-02,  5.0528e-02,  1.7700e-01, -1.7319e-02, -1.5598e-01,
           2.3484e-02,  2.1313e-01, -1.2317e-01,  1.6858e-01, -8.4938e-02,
           3.8902e-02,  7.2624e-02,  1.5431e-02, -1.7783e-01, -6.7370e-02,
          -2.7734e-01, -2.6031e-01, -1.4586e-02, -1.9599e-01,  2.5277e-02],
         [ 6.3155e-02, -4.9110e-02, -9.1053e-03,  2.1682e-01,  2.7802e-01,
           6.1583e-02,  2.2054e-01,  1.7254e-01,  2.8669e-01,  3.4806e-01,
          -2.1079e-01,  7.8890e-02, -7.6848e-02,  1.2039e-01,  6.7348e-02,
           2.0514e-01, -4.4512e-02,  2.5517e-01, -2.0666e-01, -4.9877e-02,
           6.9318e-02,  1.4547e-01, -8.1818e-03,  1.8031e-01,  1.0990e-01,
           2.4748e-01,  1.2934e-01, -7.2093e-02,  4.7949e-05,  1.0061e-01]],
        device='cuda:0', requires_grad=True), Parameter containing:
 tenso

In [0]:
test_accuracies

[0.644,
 0.683,
 0.696,
 0.698,
 0.698,
 0.697,
 0.699,
 0.7,
 0.702,
 0.704,
 0.704,
 0.707,
 0.711,
 0.71,
 0.711,
 0.715,
 0.715,
 0.713,
 0.712,
 0.714,
 0.714,
 0.713,
 0.714,
 0.714,
 0.714,
 0.714,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716,
 0.716]

In [0]:
train_losses

[0.6242465376853943,
 0.5993843078613281,
 0.5838264226913452,
 0.5749411582946777,
 0.569919764995575,
 0.5668626427650452,
 0.5646817088127136,
 0.5628995895385742,
 0.5613881945610046,
 0.5601444244384766,
 0.5591697096824646,
 0.5584392547607422,
 0.5579067468643188,
 0.5575144290924072,
 0.5572090148925781,
 0.5569486618041992,
 0.5567140579223633,
 0.5564984083175659,
 0.5563021898269653,
 0.5561279654502869,
 0.5559759736061096,
 0.5558468699455261,
 0.5557408928871155,
 0.5556522607803345,
 0.5555813908576965,
 0.5555256009101868,
 0.555484414100647,
 0.5554564595222473,
 0.5554373264312744,
 0.5554253458976746,
 0.555419921875,
 0.5554186701774597,
 0.5554169416427612,
 0.5554168820381165,
 0.5554169416427612,
 0.5554167628288269,
 0.5554167628288269,
 0.5554169416427612,
 0.5554168820381165,
 0.5554169416427612,
 0.5554160475730896,
 0.5554168820381165,
 0.5554169416427612,
 0.5554169416427612,
 0.555417001247406,
 0.5554168820381165,
 0.555417001247406,
 0.5554172992706299,


In [0]:
#t = text_list_to_pandas_nums(df_ST_s140_train_bayes, vectorizer)
total = 0

for i in range(0, 10000):
    if np.max(t.iloc[i][0:980]) == 0:
        total += 1

print(total)


680


In [0]:
!zip -r all_tokenized_data.zip tokenized_*

  adding: tokenized_bayes_train_from_imdb_file_.csv (deflated 97%)
  adding: tokenized_bayes_train_from_s140_file_.csv (deflated 99%)
  adding: tokenized_roberta_train_from_imdb_file_.csv (deflated 97%)
  adding: tokenized_roberta_train_from_s140_file_.csv (deflated 99%)
  adding: tokenized_test_from_amazon_file_.csv (deflated 99%)
  adding: tokenized_test_from_imdb_file_.csv (deflated 97%)
  adding: tokenized_test_from_s140_file_.csv (deflated 99%)
  adding: tokenized_test_from_yelp_file_.csv (deflated 99%)


In [0]:
!rm -rf /content/tokenized*

In [0]:
bayes_trains = pdframe_s140_train_bayes.append(pdframe_imdb_train_bayes)
mi_scores = np.zeros(len(bayes_trains.columns) - 2)



for i in range(0, len(bayes_trains.columns) - 2):
    mi_scores[i] = sklearn.metrics.mutual_info_score(bayes_trains[i], bayes_trains['class'])

In [0]:
mi_scores

array([1.56769936e-05, 2.07992477e-04, 1.06105391e-05, 9.29781570e-05,
       8.01376818e-05, 1.42888736e-04, 5.41534842e-05, 1.07608562e-05,
       1.36683095e-04, 1.92053760e-04, 1.82109619e-04, 1.83030540e-04,
       1.96825168e-05, 1.35070582e-03, 7.31659927e-05, 1.46151687e-05,
       1.52915020e-04, 4.87265342e-05, 8.28557971e-05, 3.08963073e-04,
       3.98623424e-04, 6.51929625e-05, 1.46960696e-05, 2.22027424e-04,
       2.20250601e-04, 6.14478608e-05, 7.03943600e-05, 2.35631668e-03,
       8.45746579e-05, 1.82189086e-04, 2.22754318e-04, 4.12352670e-06,
       1.97698696e-04, 9.46915484e-04, 9.99213833e-04, 1.26828436e-04,
       4.98681372e-04, 7.48314901e-05, 6.94922210e-05, 3.53460347e-04,
       1.47578496e-04, 1.10014049e-04, 4.94664887e-05, 2.62463712e-04,
       7.14321377e-04, 1.05209378e-04, 4.94762374e-05, 3.36280722e-05,
       2.04282078e-04, 1.18134991e-05, 1.75336515e-03, 6.22478074e-04,
       2.24293494e-03, 4.37916209e-03, 6.10881646e-05, 7.90240058e-05,
      